## Overview
Keytool is a utility included in JDK to do various operations around certificates and keys.

## Keystore
Keystore is a file which saves certificates, private/ public keys and secret keys. Since Java 9, the default keystore format is `PKCS12`. The biggest difference between `JKS` and `PKCS12` is that `JKS` is a format specific to Java, while `PKCS12` is a standardized and language-neutral way of storing encrypted private keys and certificates.

JDK maintains a CA keystore file named `cacerts` in folder `jre/lib/security`. By default, the keytool utility looks for keystore file named `.keystore` in the home directory `$HOME/.keystore`

Keystore files are password protected, the default password for `cacerts` is `changeit`.

If we do not specify the `-keystore` flag to keytool, it uses the default keystore, else it uses the specified keystore:
```bash
~$ keytool -keystore mykeystore.jks -list
```

## List Entries
To view all entries in the keystore, use the `-list` option:
```bash
~$ keytool -keystore "$JAVA_HOME/lib/security/cacerts" -list
Warning: use -cacerts option to access cacerts keystore
Enter keystore password:
Keystore type: JKS
Keystore provider: SUN

Your keystore contains 89 entries

actalisauthenticationrootca [jdk], 22-Sep-2011, trustedCertEntry,Certificate fingerprint (SHA-256): 55:92:60:84:EC:96:3A:64:B9:6E:2A:BE:01:CE:0B:A8:6A:64:FB:FE:BC:C7:AA:B5:AF:C1:55:B3:7F:D7:60:66
addtrustexternalca [jdk], 30-May-2000, trustedCertEntry,Certificate fingerprint (SHA-256): 68:7F:A4:51:38:22:78:FF:F0:C8:B1:1F:8D:43:D5:76:67:1C:6E:B2:BC:EA:B4:13:FB:83:D9:65:D0:6D:2F:F2
```

There is an alternate way to access `cacerts` keystore:
```bash
~$ keytool -cacerts -list
```

To avoid manually typing in password:
```bash
~$ keytool -cacerts -list -storepass changeit
```

## Alias
All keystore entries (key and trusted certificate entries) are accessed via unique aliases. Aliases are not single-object identifiers; you may use the same alias for a key and a cert entry in a keystore, without one wiping out the other. If alias is not specified, it is assumed to be `mykey`.

## Generating
We can use the following commands to generate:
- `-genkeypair`   Generates a key pair
- `-genseckey`    Generates a secret key
- `-gencert`      Generates certificate from a certificate request
- `-certreq`      Generate cert request

### Generate Keypair
This command generates a private/public keypair along with a self-signed certicate wrapping the public key.
```bash
~$ keytool -keystore mykeystore.jks -storepass mykeystorepass -genkeypair -alias mykeypair -keyalg RSA -keysize 2048 -validity 90
```
`-keysize` and `-validity` are not mandatory. On executing the above command, it asks for Distinguished Name information to bundle in the self-signed certificate:
```
What is your first and last name?
  [Salman Anjum]:
What is the name of your organizational unit?
  [Unknown]:
What is the name of your organization?
  [Unknown]:
What is the name of your City or Locality?
  [Unknown]:
What is the name of your State or Province?
  [Unknown]:
What is the two-letter country code for this unit?
  [Unknown]:
Is CN=Salman Anjum, OU=Unknown, O=Unknown, L=Unknown, ST=Unknown, C=Unknown correct?
  [no]:  y
```
We can simplify it by adding `-dname` option, for example:
```
-dname "CN=John Smith, OU=Development, O=Standard Supplies Inc., L=Anytown, S=North Carolina, C=US"
```
Another option is `-keypass` which lets us password protect the private key. If not specified, we need to pass in the keystore password to access the private key.

### Delete a Key
We can delete a key by its alias like this:
```bash
~$ keytool -keystore mykeystore.jks -storepass mykeystorepass -delete -alias mykeypair
```

### Export
To export the .pem certificate (in X.509 format):
```bash
~$ keytool -keystore mykeystore.jks -storepass mykeystorepass -exportcert -alias mykeypair -file mycert.pem -rfc
```

## Import
Keytool allows to import the following:
- `-importcert`         Imports a certificate or a certificate chain
- `-importpass`         Imports a password
- `-importkeystore`     Imports one or all entries from another keystore

```bash
~$ keytool -importcert -file mycert.pem -alias myotherkey
```

## Truststore
The main difference between truststore vs keystore is that truststore (as the name suggest) is used to store certificates from trusted Certificate Authorities (CA) which are used to verify certificate presented by server in SSL Connection while keystore is used to store the private key and own identity certificate which program should present to other parties (server or client) to verify its identity (for example in TLS handshake). 

The `cacerts` keystore presented earlier is an example of truststore.